In [1]:
import os
import sys
import time
import pickle
import string
from datetime import datetime
from datetime import timedelta
from itertools import chain, cycle
import collections
from pprint import pprint
from io import StringIO
import pathlib
import sqlite3

import numpy as np
import pandas as pd
import dask.dataframe as dd
from pandas.tools.plotting import parallel_coordinates
import matplotlib.pyplot as plt

#import urllib
#import webbrowser
#import codecs
from bs4 import BeautifulSoup
from xml.dom import minidom
from xml.etree import ElementTree as ET
#from lxml import etree as ET #Supports xpath syntax
import xmltodict 

import nltk
import re
import Levenshtein
import networkx as nx
import json

#from nominatim import Nominatim
#from mpl_toolkits.basemap import Basemap   #http://matplotlib.org/basemap/api/basemap_api.html
from matplotlib import cm
import geopy.geocoders as gg
from geopy.geocoders import Nominatim
import geopandas as gpd
import ipyleaflet
#from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, WMSLayer, Marker, MarkerCluster

##https://python-visualization.github.io/folium/docs-v0.6.0/
#https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf
import folium
import folium.plugins
#from folium.plugins import MarkerCluster
#from folium.plugins import FastMarkerCluster


#import plotly as py
#import plotly.graph_objs as go
#import plotly.figure_factory as ff

from IPython.display import Image, HTML, IFrame, FileLink, FileLinks #needed to render in notebook
from IPython.core.display import display

#import pydotplus #Install http://www.graphviz.org/ & Instal https://pypi.python.org/pypi/pydotplus

#https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary-in-python
#https://github.com/martinblech/xmltodict
#https://www.datacamp.com/community/tutorials/python-xml-elementtree
#https://stackabuse.com/reading-and-writing-xml-files-in-python/
#https://codereview.stackexchange.com/questions/117810/parsing-of-xml-in-python-using-minidom-and-inserting-data-into-a-database
#https://stackoverflow.com/questions/749796/pretty-printing-xml-in-python
#https://hackersandslackers.com/dealing-with-xml-in-python/

In [2]:
pwd

'C:\\Users\\rsouza\\Documents\\Python Scripts\\exploreAT-TEI_XML_NLP'

### PART II: Extracting information from XML data files

#### Reading a single XML Data file to test:

In [3]:
xmlfile =  './TEI-XML-2018/bel_d-tei-02/d157_qdb-TEI-02.xml'
#xmlfile = open(xmlfile, mode='r', encoding='utf-8').read()
namespaces = {'TEI': 'http://www.tei-c.org/ns/1.0'} # add more as needed

Parsing the XML tree

In [4]:
xmldoc = minidom.parse(xmlfile)  #xmldoc = ET.XML(xmlfile)
bs = BeautifulSoup(open(xmlfile, encoding="utf8"), 'xml')
tree = ET.parse(xmlfile)

In [5]:
root = tree.getroot()
print(root)

<Element '{http://www.tei-c.org/ns/1.0}TEI' at 0x000002BEB16BE6D8>


In [6]:
my_namespaces = dict([node for _, node in ET.iterparse(StringIO(open(xmlfile, mode='r', encoding='utf-8').read()), events=['start-ns'])])
print(my_namespaces)

{'': 'http://www.tei-c.org/ns/1.0'}


In [7]:
for child in root:
    print(child.tag, child.attrib)

{http://www.tei-c.org/ns/1.0}teiHeader {}
{http://www.tei-c.org/ns/1.0}text {}


In [8]:
set([elem.tag for elem in root.iter()])

{'{http://www.tei-c.org/ns/1.0}TEI',
 '{http://www.tei-c.org/ns/1.0}att',
 '{http://www.tei-c.org/ns/1.0}bibl',
 '{http://www.tei-c.org/ns/1.0}body',
 '{http://www.tei-c.org/ns/1.0}certainty',
 '{http://www.tei-c.org/ns/1.0}cit',
 '{http://www.tei-c.org/ns/1.0}date',
 '{http://www.tei-c.org/ns/1.0}def',
 '{http://www.tei-c.org/ns/1.0}edition',
 '{http://www.tei-c.org/ns/1.0}editionStmt',
 '{http://www.tei-c.org/ns/1.0}editorialDecl',
 '{http://www.tei-c.org/ns/1.0}encodingDesc',
 '{http://www.tei-c.org/ns/1.0}entry',
 '{http://www.tei-c.org/ns/1.0}etym',
 '{http://www.tei-c.org/ns/1.0}fileDesc',
 '{http://www.tei-c.org/ns/1.0}form',
 '{http://www.tei-c.org/ns/1.0}geoDecl',
 '{http://www.tei-c.org/ns/1.0}gi',
 '{http://www.tei-c.org/ns/1.0}gram',
 '{http://www.tei-c.org/ns/1.0}gramGrp',
 '{http://www.tei-c.org/ns/1.0}idno',
 '{http://www.tei-c.org/ns/1.0}interp',
 '{http://www.tei-c.org/ns/1.0}label',
 '{http://www.tei-c.org/ns/1.0}listPlace',
 '{http://www.tei-c.org/ns/1.0}listPrefixDe

In [9]:
#for elem in root.iter():
for elem in root.iter('{http://www.tei-c.org/ns/1.0}entry'):
    for attrName, attrValue in elem.items():
        print(attrName + ' ==> ' + attrValue)
    #print(elem.tag)
    #print(elem.text)
    #print(elem.attrib)
    #print(elem.items())
    #print('\n')

{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e2
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e43
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e70
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e91
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e110
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e131
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e152
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e179
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e209
{http://www.w3.org/XML/1998/namespace}lang ==> b

#### Iterating through all XML Data files

In [10]:
datapath = './TEI-XML-2018/'
onlydirs = [f for f in os.listdir(datapath) if os.path.isdir(os.path.join(datapath,f)) and f.startswith('bel')]
onlydirs

['bel_d-tei-02',
 'bel_e-tei-02',
 'bel_f-tei-02',
 'bel_g-tei-02',
 'bel_h-tei-02',
 'bel_i-j-tei-02',
 'bel_k-tei-02',
 'bel_l-tei-02',
 'bel_m-tei-02',
 'bel_n-o-p-tei-02',
 'bel_q-r-tei-02',
 'bel_s-tei-02',
 'bel_u-w-tei-02',
 'bel_z-tei-02']

In [11]:
onlyfiles = []
for folder in onlydirs:
    for filex in [f for f in os.listdir(os.path.join(datapath, folder)) if os.path.isfile(os.path.join(datapath, folder, f)) \
                                       and not f.startswith('.')]:
        onlyfiles.append('{}{}/{}'.format(datapath, folder, filex))


print('Files in the folder:')
for i, w in enumerate(onlyfiles):
    print(i+1, '--' ,w)

Files in the folder:
1 -- ./TEI-XML-2018/bel_d-tei-02/d154_156_qdb-TEI-02.xml
2 -- ./TEI-XML-2018/bel_d-tei-02/d157_qdb-TEI-02.xml
3 -- ./TEI-XML-2018/bel_d-tei-02/d158_qdb-TEI-02.xml
4 -- ./TEI-XML-2018/bel_d-tei-02/d159_qdb-TEI-02.xml
5 -- ./TEI-XML-2018/bel_d-tei-02/d160_qdb-TEI-02.xml
6 -- ./TEI-XML-2018/bel_d-tei-02/d161_qdb-TEI-02.xml
7 -- ./TEI-XML-2018/bel_d-tei-02/d162_qdb-TEI-02.xml
8 -- ./TEI-XML-2018/bel_d-tei-02/d163_qdb-TEI-02.xml
9 -- ./TEI-XML-2018/bel_d-tei-02/d164_qdb-TEI-02.xml
10 -- ./TEI-XML-2018/bel_d-tei-02/d165_qdb-TEI-02.xml
11 -- ./TEI-XML-2018/bel_d-tei-02/d166_qdb-TEI-02.xml
12 -- ./TEI-XML-2018/bel_d-tei-02/d167_qdb-TEI-02.xml
13 -- ./TEI-XML-2018/bel_d-tei-02/d168_qdb-TEI-02.xml
14 -- ./TEI-XML-2018/bel_d-tei-02/d169_qdb-TEI-02.xml
15 -- ./TEI-XML-2018/bel_d-tei-02/d170_qdb-TEI-02.xml
16 -- ./TEI-XML-2018/bel_d-tei-02/d171_qdb-TEI-02.xml
17 -- ./TEI-XML-2018/bel_d-tei-02/d172_qdb-TEI-02.xml
18 -- ./TEI-XML-2018/bel_d-tei-02/d173_qdb-TEI-02.xml
19 -- ./TEI-

In [13]:
len(list_records)

2416499

In [14]:
df_lemmas = pd.DataFrame(list_records)

In [73]:
df_lemmas = pd.read_pickle('./df_lemmas.pkl')

In [74]:
df_lemmas.info(verbose=True, null_counts=True, memory_usage='deep') #, max_cols=100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2416499 entries, 0 to 2416498
Data columns (total 46 columns):
Bundesland               1316889 non-null object
Bundesland_idno          1316889 non-null object
Gemeinde                 1198447 non-null object
Gemeinde_idno            0 non-null object
Großregion               1296722 non-null object
Großregion_idno          1225463 non-null object
Kleinregion              1286463 non-null object
Kleinregion_idno         1209145 non-null object
Ort                      1198447 non-null object
Ort_idno                 1198447 non-null object
archiv                   2416419 non-null object
bibl                     1719656 non-null object
certainty                2416499 non-null object
cit_def                  2416499 non-null object
cit_interp               2416499 non-null object
cit_note                 2416499 non-null object
cit_pRef                 2416499 non-null object
cit_quote                2416499 non-null object
cit_re     

In [75]:
df_lemmas.iloc[1]

Bundesland                                                            None
Bundesland_idno                                                       None
Gemeinde                                                              None
Gemeinde_idno                                                         None
Großregion                                                            None
Großregion_idno                                                       None
Kleinregion                                                           None
Kleinregion_idno                                                      None
Ort                                                                   None
Ort_idno                                                              None
archiv                               HK 154, d154^#2.1 = d1540308.pir^#2.1
bibl                                                                  None
certainty                                                               []
cit_def                  

In [76]:
for entry in df_lemmas.columns:
    df_lemmas[entry] = df_lemmas[entry].apply(lambda x:x if x != [] else None)

In [77]:
df_lemmas['tustep'] = df_lemmas['tustep'].apply(lambda x:x if x == None else tuple(x))
df_lemmas['etym'] = df_lemmas['etym'].apply(lambda x:x if x == None else tuple(x))
df_lemmas['cit_def'] = df_lemmas['cit_def'].apply(lambda x:x if x == None else tuple(x))
df_lemmas['cit_interp'] = df_lemmas['cit_interp'].apply(lambda x:x if x == None else tuple(x))
df_lemmas['cit_note'] = df_lemmas['cit_note'].apply(lambda x:x if x == None else tuple(x))
df_lemmas['cit_pRef'] = df_lemmas['cit_pRef'].apply(lambda x:x if x == None else tuple(x))
df_lemmas['cit_quote'] = df_lemmas['cit_quote'].apply(lambda x:x if x == None else tuple(x))
df_lemmas['cit_re'] = df_lemmas['cit_re'].apply(lambda x:x if x == None else tuple(x))
df_lemmas['cit_ref'] = df_lemmas['cit_ref'].apply(lambda x:x if x == None else tuple(x))
df_lemmas['cit_type'] = df_lemmas['cit_type'].apply(lambda x:x if x == None else tuple(x))
df_lemmas['certainty'] = df_lemmas['certainty'].apply(lambda x:x if x == None else tuple(x))

In [78]:
del df_lemmas['Gemeinde_idno']

In [13]:
df_lemmas.info(verbose=True, null_counts=True, memory_usage='deep') #, max_cols=100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2416499 entries, 0 to 2416498
Data columns (total 45 columns):
Bundesland               1316889 non-null object
Bundesland_idno          1316889 non-null object
Gemeinde                 1198447 non-null object
Großregion               1296722 non-null object
Großregion_idno          1225463 non-null object
Kleinregion              1286463 non-null object
Kleinregion_idno         1209145 non-null object
Ort                      1198447 non-null object
Ort_idno                 1198447 non-null object
archiv                   2416419 non-null object
bibl                     1719656 non-null object
certainty                25631 non-null object
cit_def                  461023 non-null object
cit_interp               353677 non-null object
cit_note                 111690 non-null object
cit_pRef                 79913 non-null object
cit_quote                704560 non-null object
cit_re                   113327 non-null object
cit_ref       

In [14]:
df_lemmas.columns

Index(['Bundesland', 'Bundesland_idno', 'Gemeinde', 'Großregion',
       'Großregion_idno', 'Kleinregion', 'Kleinregion_idno', 'Ort', 'Ort_idno',
       'archiv', 'bibl', 'certainty', 'cit_def', 'cit_interp', 'cit_note',
       'cit_pRef', 'cit_quote', 'cit_re', 'cit_ref', 'cit_type', 'date',
       'entry_id', 'etym', 'filename', 'fragebogenNummer',
       'hauptlemma_normalized', 'hauptlemma_orig', 'hauptlemma_unique',
       'nebenlemma_normalized', 'nebenlemma_orig', 'nebenlemma_unique',
       'note_anmerkung', 'note_diverse', 'note_notabene', 'orig_placeName',
       'paragraph', 'placeName_id', 'pos', 'quelle', 'quelleBearbeitet',
       'seite', 'simple_placeName', 'tustep', 'usg_corresp', 'verweise'],
      dtype='object')

In [15]:
entry_info = ['filename', 'entry_id']

regions_info = ['Bundesland', 
                'Bundesland_idno', 
                'Gemeinde', 
                #'Gemeinde_idno',
                'Großregion', 
                'Großregion_idno', 
                'Kleinregion', 
                'Kleinregion_idno',
                'Ort', 
                'Ort_idno',
                'simple_placeName',
                'orig_placeName',
                'placeName_id', 
                'usg_corresp',]

lemmas_info = ['date',
               'hauptlemma_normalized', 
               'hauptlemma_orig', 
               'hauptlemma_unique',
               'nebenlemma_normalized',
               'nebenlemma_orig', 
               'nebenlemma_unique', 
               'tustep',
               'etym', 
               'pos',]

ref_info = ['verweise',
            'archiv',
            'quelle',
            'quelleBearbeitet', 
            'seite', 
            'fragebogenNummer', 
            'paragraph', 
            'bibl',]

cit_info = ['cit_def',
            'cit_interp',
            'cit_note',
            'cit_pRef',
            'cit_quote',
            'cit_re',
            'cit_ref',
            'cit_type', 
            'certainty',]

note_info = ['note_anmerkung',
             'note_diverse',
             'note_notabene',]

In [16]:
df_lemmas.head(10)

,Bundesland,Bundesland_idno,Gemeinde,Großregion,Großregion_idno,Kleinregion,Kleinregion_idno,Ort,Ort_idno,archiv,...,paragraph,placeName_id,pos,quelle,quelleBearbeitet,seite,simple_placeName,tustep,usg_corresp,verweise
0,None,None,None,None,None,None,None,None,None,"HK 154, d154^#1.1 = d1540308.pir^#1.1",...,None,None,Subst,Egger Gl.,"{1B,1C} hist. TirWeistGl.",30,None,"(diaun,)",None,None
1,None,None,None,None,None,None,None,None,None,"HK 154, d154^#2.1 = d1540308.pir^#2.1",...,None,None,Subst,Schneider Tir.(1965),"{1B,1C:X} Etym.^@ SCHNEIDER· (1963)",601,Chur,"(diaun,)",None,None
2,None,None,None,None,None,None,None,None,None,"HK 154, d154^#3.1 = d1540308.pir^#3.1",...,None,None,Subst,Tir.Wb.,"{1B,1C} Tir. TirWb.(1955-1956)","1,1293,337, f337^#3.4 = f.",Chur,"(diaun,)",None,None
3,None,None,None,None,None,None,None,None,None,"HK 154, d154^#4.1 = d1540308.pir^#4.1",...,None,None,Interj,Straßburg Umg. Nagelmayer,"{2.2d,2.3b,2.3c,2.3d} Gurkt.:MKä.:Kä. FbB.NAG...",None,None,"(tib,)",None,None
4,None,None,None,None,None,None,None,None,None,"HK 154, d154^#5.1 = d1540308.pir^#5.1",...,None,None,Interj,Straßburg Umg. Nagelmayer,"{2.2d,2.3b,2.3c,2.3d} Gurkt.:MKä.:Kä. FbB.NAG...",None,None,"(tib,)",None,None
5,None,None,None,None,None,None,None,None,None,"HK 154, d154^#6.1 = d1540308.pir^#6.1",...,None,None,Interj,Straßburg Umg. Nagelmayer,"{2.2d,2.3b,2.3c,2.3d} Gurkt.:MKä.:Kä. FbB.NAG...",None,None,"(tibo, tibå)",None,None
6,St.,3,Lobmingtal,sbair.ObSt.,3.1,mittl.obMurt.,3.1d,Kleinlobming,3.1d03,"HK 154, d154^#7.1 = d1540308.pir^#7.1",...,None,sigle:3.1d03,Interj,Gr.Lobming Geosich,{3.1d03} mittl.obMurt.:obMurt.:mObMurgeb.:ObMu...,None,None,"(tib!,)",None,None
7,None,None,None,None,None,None,None,None,None,"HK 154, d154^#8.1 = d1540308.pir^#8.1",...,None,None,Subst,Lexer,{X} MhdWb./L.,"1,441",None,"(diubde,)",None,s.
8,None,None,None,None,None,None,None,None,None,"HK 154, d154^#9.1 = d1540308.pir^#9.1",...,None,None,Subst,Wilmanns Dt.Gramm.,{X} DGr./W.,"2,343",None,"(diubde,)",None,None
9,None,None,None,None,None,None,None,None,None,"HK 154, d154^#10.1 = d1540308.pir^#10.1",...,None,None,Subst,Grimm DWB²,{X} DWB,"6,9296,58",None,"(Diebde,)",None,None


In [17]:
for column in df_lemmas[regions_info]:
    print('unique values for column {}'.format(column))
    #print(df_lemmas[column].unique())
    print(df_lemmas[column].value_counts(dropna=False))
    print()

unique values for column Bundesland
NaN      1099610
NÖ        412316
OÖ        273038
NTir.     175406
St.       140391
Kä.        96426
Sa.        86259
STir.      72034
Bgl.       44676
OTir.      16343
Name: Bundesland, dtype: int64

unique values for column Bundesland_idno
NaN    1099610
6       412316
5       273038
1C      175406
3       140391
2        96426
4        86259
1A       72034
7        44676
1B       16343
Name: Bundesland_idno, dtype: int64

unique values for column Gemeinde
NaN                                1218052
Wien                                 71263
Mühlheim am Inn                      34694
Gmunden                              33056
Kautzen                              32858
Kufstein                             21403
Linz                                 21281
Thiersee                             20598
Poysdorf                             19640
Rangersdorf                          16430
Stainz                               16174
Traun                      

In [18]:
for column in df_lemmas[lemmas_info]:
    print('unique values for column {}'.format(column))
    #print(df_lemmas[column].unique())
    print(df_lemmas[column].value_counts(dropna=False))
    print()

unique values for column date
NaN                      2351439
17.Jh.                       738
16.Jh.                       707
1587                         634
1623                         631
15.Jh.                       567
16. Jh.                      498
1550                         486
17. Jh.                      486
1581                         462
1532                         455
1969                         431
1463                         400
1368                         388
15. Jh.                      377
1498                         345
1465                         329
1553                         319
1367                         311
1590                         304
1454                         304
1573                         302
1467                         292
1494                         291
1365                         287
1460                         274
1621                         262
1624                         260
1370                         257
1514         

In [19]:
for column in df_lemmas[ref_info]:
    print('unique values for column {}'.format(column))
    #print(df_lemmas[column].unique())
    print(df_lemmas[column].value_counts(dropna=False))
    print()

unique values for column verweise
NaN                                                    2234830
\n                                                       80246
s.                                                       26831
\n                                                       13641
vgl.                                                      5853
=                                                         4561
s. a.                                                     2494
auch                                                      2196
Syn.                                                      2055
auch:                                                     1021
oder                                                       992
siehe                                                      779
od.                                                        712
Syn.:                                                      645
zu                                                         562
SYN.                 

In [20]:
for column in df_lemmas[cit_info]:
    print('unique values for column {}'.format(column))
    #print(df_lemmas[column].unique())
    print(df_lemmas[column].value_counts(dropna=False))
    print()

unique values for column cit_def
NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   1955476
(--,)                                                                                                                                                                                                                                                                                                                                                                                    

In [94]:
df_lemmas.to_pickle('./df_lemmas.pkl')

#### Building a SQLite Database

- From Dataframe

In [12]:
df_lemmas = pd.read_pickle('./df_lemmas.pkl')

In [101]:
conn = sqlite3.connect('./lemmas_db1.sqlite')

In [97]:
df_lemmas.applymap(str).head()

,Bundesland,Bundesland_idno,Gemeinde,Großregion,Großregion_idno,Kleinregion,Kleinregion_idno,Ort,Ort_idno,archiv,...,paragraph,placeName_id,pos,quelle,quelleBearbeitet,seite,simple_placeName,tustep,usg_corresp,verweise
0,None,None,None,None,None,None,None,None,None,"HK 154, d154^#1.1 = d1540308.pir^#1.1",...,None,None,Subst,Egger Gl.,"{1B,1C} hist. TirWeistGl.",30,None,"('diaun',)",None,None
1,None,None,None,None,None,None,None,None,None,"HK 154, d154^#2.1 = d1540308.pir^#2.1",...,None,None,Subst,Schneider Tir.(1965),"{1B,1C:X} Etym.^@ SCHNEIDER· (1963)",601,Chur,"('diaun',)",None,None
2,None,None,None,None,None,None,None,None,None,"HK 154, d154^#3.1 = d1540308.pir^#3.1",...,None,None,Subst,Tir.Wb.,"{1B,1C} Tir. TirWb.(1955-1956)","1,1293,337, f337^#3.4 = f.",Chur,"('diaun',)",None,None
3,None,None,None,None,None,None,None,None,None,"HK 154, d154^#4.1 = d1540308.pir^#4.1",...,None,None,Interj,Straßburg Umg. Nagelmayer,"{2.2d,2.3b,2.3c,2.3d} Gurkt.:MKä.:Kä. FbB.NAG...",None,None,"('tib',)",None,None
4,None,None,None,None,None,None,None,None,None,"HK 154, d154^#5.1 = d1540308.pir^#5.1",...,None,None,Interj,Straßburg Umg. Nagelmayer,"{2.2d,2.3b,2.3c,2.3d} Gurkt.:MKä.:Kä. FbB.NAG...",None,None,"('tib',)",None,None


In [102]:
df_lemmas.applymap(str).to_sql(name='lemmas', con=conn)

In [103]:
conn.close()

- From list of dicts

In [24]:
with open('./list_lemmas.pkl', 'rb') as f:
    list_lemmas = pickle.load(f)

In [25]:
list_lemmas[0]

{'filename': './TEI-XML-2018/bel_d-tei-02/d154_156_qdb-TEI-02.xml',
 'entry_id': 'd154_156_qdb-d1e2',
 'date': None,
 'usg_corresp': None,
 'simple_placeName': None,
 'orig_placeName': None,
 'placeName_id': None,
 'Bundesland': None,
 'Bundesland_idno': None,
 'Großregion': None,
 'Großregion_idno': None,
 'Kleinregion': None,
 'Kleinregion_idno': None,
 'Gemeinde': None,
 'Gemeinde_idno': None,
 'Ort': None,
 'Ort_idno': None,
 'hauptlemma_unique': 'Diaun',
 'hauptlemma_orig': None,
 'hauptlemma_normalized': None,
 'nebenlemma_unique': None,
 'nebenlemma_orig': None,
 'nebenlemma_normalized': None,
 'verweise': None,
 'archiv': 'HK 154, d154^#1.1 = d1540308.pir^#1.1',
 'quelle': 'Egger Gl.',
 'quelleBearbeitet': '{1B,1C} hist.  TirWeistGl.',
 'seite': '30',
 'fragebogenNummer': None,
 'paragraph': None,
 'bibl': None,
 'tustep': ['diaun'],
 'pos': 'Subst',
 'etym': ['= rätorom. Entspr. f.', 'Dëchant', 'Dekan'],
 'note_notabene': None,
 'note_anmerkung': 'Einzelbelege aus/zu d154-d156

In [28]:
conn = sqlite3.connect('./lemmas_db.sqlite')
cur = conn.cursor()

In [30]:
q = '''
CREATE TABLE IF NOT EXISTS lemmas (
Gemeinde_idno text,
hauptlemma_orig text,
quelle text,
bibl text,
pos text,
Kleinregion_idno text,
cit_type text,
etym text,
cit_quote text,
verweise text,
Bundesland_idno text,
tustep text,
note_diverse text,
orig_placeName text,
note_anmerkung text,
nebenlemma_unique text,
Großregion text,
quelleBearbeitet text,
archiv text,
Kleinregion text,
placeName_id text,
seite text,
Gemeinde text,
Ort_idno text,
cit_note text,
Ort text,
cit_re text,
hauptlemma_unique text,
cit_interp text,
cit_def text,
Bundesland text,
date text,
nebenlemma_normalized text,
hauptlemma_normalized text,
paragraph text,
entry_id text,
nebenlemma_orig text,
certainty text,
Großregion_idno text,
cit_ref text,
fragebogenNummer text,
note_notabene text,
usg_corresp text,
filename text,
simple_placeName text,
cit_pRef text
)
'''

In [31]:
# Create table
cur.execute(q)

In [32]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('lemmas',)]


In [33]:
cur.execute('select count(*) from lemmas')
print(cur.fetchall())

[(0,)]


In [34]:
def post_row(conn, tablename, rec):
    keys = ','.join(rec.keys())
    question_marks = ','.join(list('?'*len(rec)))
    values = [str(v) for v in rec.values()]
    values = tuple(values)
    conn.execute('INSERT INTO '+tablename+' ('+keys+') VALUES ('+question_marks+')', values)

In [35]:
for idno, entry in enumerate(list_lemmas):
    post_row(conn, 'lemmas', entry)
    if idno%100000 == 0:
        print(idno)
conn.commit()
#conn.close()

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
